# Used car auction prices dataset assignment

#### Research Question: 
    As a car salesman, how can you effectively predict car sales prices based on features such as color, year, odometer reading, and condition, thereby facilitating our business in making informed pricing decisions?

#### Sub Questions: 
    1. How do the features impact the accuracy of predictions from the Random Forest, ANN, KNN, XGBoost, and Linear Regression models?
    2. What hyperparameter configurations for each model (Random Forest, ANN, KNN, XGBoost) result in the best predictive accuracy for car sales prices?
    3. How do the Random Forest, ANN, KNN, XGBoost, and Linear Regression models demonstrate robustness in predicting car sales prices?
    4. How do the Random Forest, ANN, KNN, XGBoost, and Linear Regression models compare in terms of their predictive performance for car sales prices, and can we identify specific scenarios or feature patterns where one model consistently outperforms the others?
    

## 1. Exploratory Data Analysis

### 1.1 Import packages

In [ ]:
import numpy as np
import pandas as pd
from plotnine import * 
import os
import scipy
from scipy import stats
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
from IPython.core.display import HTML 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from tabulate import tabulate
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import mean_squared_error, r2_score
from scikeras.wrappers import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense
import datetime
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression

### 1.2 Read data from path

An extra column is loaded due to having data but no column name. 
This is because of an error of data in the dataset and will be adjusted after.

In [ ]:
# Get the current working directory
cwd = os.getcwd()

# List all the directories and files in the parent directory of 'dataset'
dir_list = os.listdir(os.path.join(cwd, ''))

car_prices_data_path = os.path.join( "", "car_prices.csv")

# Read csv file and name column to make sure the 17th column is included
df = pd.read_csv(car_prices_data_path, header=None, names=[f'col{i}' for i in range(1, 18)])

# Give the first row of this column a value
df.iloc[0, 16] = 'extra'

# Now, use the first row as column names
df.columns = df.iloc[0]

# Drop the first row since it's now redundant as column names
df = df[1:]

### 1.3 Examine and Clean the data

The unalligned data is alligned into the correct columns

In [ ]:
# Filter rows with data in the 'extra' column
df_extra = df[df['extra'].notnull()]

# Include columns starting from column 5 onwards
df_extra = df_extra.iloc[:, 5:]

# Replace column names with the name of the column to the right
df_extra.columns = df_extra.columns.to_series().shift(+1).fillna('body')

# Iterate over the rows of df_extra and update values in df
for index, row in df_extra.iterrows():
    for column_name in df.columns:
        # Update values in df only for shared rows and columns
        if column_name in df_extra.columns and index in df.index:
            df.at[index, column_name] = row[column_name]

# drop the 'extra' column from the dataframe
df = df.drop('extra', axis=1)

Change data types

In [ ]:
# Convert columns to appropriate data types
df['year'] = df['year'].astype('int')
df['odometer'] = df['odometer'].astype('float')
df['mmr'] = df['mmr'].astype('float')
df['sellingprice'] = df['sellingprice'].astype('float')
df['condition'] = df['condition'].astype('float')
df['saledate'] = pd.to_datetime(df['saledate'])

Numeric statistics

In [ ]:
# Calculate shape
print("There are {} rows and {} columns in the dataset".format(df.shape[0], df.shape[1]))

# Select only numeric columns
numeric_columns = df.select_dtypes(include=['number'])

# get statistics for numeric columns
nan_counts_numeric = numeric_columns.isnull().sum()
unique_counts_numeric = numeric_columns.nunique()
mode_counts_numeric = numeric_columns.mode().iloc[0]
max_values_numeric = numeric_columns.max()
min_values_numeric = numeric_columns.min()
percentage_nan_numeric = numeric_columns.isnull().mean() * 100
mean_values_numeric = numeric_columns.mean()
median_values_numeric = numeric_columns.median()
variance_values_numeric = numeric_columns.var()
std_dev_values_numeric = numeric_columns.std()
skewness_values_numeric = numeric_columns.skew()
kurtosis_values_numeric = numeric_columns.kurt()

# create a new dataframe with the statistics for numeric columns
numeric_statistics = pd.DataFrame({'Nan Count': nan_counts_numeric,
                                    'Unique Count': unique_counts_numeric,
                                    'Mode Count': mode_counts_numeric,
                                    'Max Value': max_values_numeric,
                                    'Min Value': min_values_numeric,
                                    'Percentage Nan': percentage_nan_numeric,
                                    'Mean Value': mean_values_numeric,
                                    'Median Value': median_values_numeric,
                                    'Variance Value': variance_values_numeric,
                                    'Standard Deviation Value': std_dev_values_numeric,
                                    'Skewness Value': skewness_values_numeric,
                                    'Kurtosis Value': kurtosis_values_numeric})

# Print the numeric statistics dataframe
print(tabulate(numeric_statistics, headers='keys', tablefmt='fancy_grid'))


Non numeric statistics

In [ ]:
# Select only non-numeric columns
non_numeric_columns = df.select_dtypes(exclude=['number'])

# get statistics for non-numeric columns
unique_counts_non_numeric = non_numeric_columns.nunique()
mode_counts_non_numeric = non_numeric_columns.mode().iloc[0]
nan_counts_non_numeric = non_numeric_columns.isnull().sum()
percentage_nan_non_numeric = non_numeric_columns.isnull().mean() * 100

# create a new dataframe with the statistics for non-numeric columns
non_numeric_statistics = pd.DataFrame({'Nan Count': nan_counts_non_numeric,
                                       'Unique Count': unique_counts_non_numeric,
                                       'Mode Count': mode_counts_non_numeric,
                                       'Percentage Nan': percentage_nan_non_numeric})

# print the non-numeric statistics dataframe
print(tabulate(non_numeric_statistics, headers='keys', tablefmt='fancy_grid'))


Filling missing values

In [ ]:
grouped = df.groupby('model')
modes = grouped['transmission'].apply(lambda x: stats.mode(x)[0][0])
df['transmission'].fillna(df['model'].map(modes), inplace=True)

# Calculate the mode of the 'transmission' column
mode = df['transmission'].mode()[0]

# Fill the remaining NaN values in the 'transmission' column with the calculated mode
df['transmission'].fillna(mode, inplace=True)

# Calculate the mode of the 'transmission' column
mode = df['transmission'].mode()[0]

# Drop all rows with NaN values in the other rows
df.dropna(inplace=True)

Check if NaN values are filtered out and shape of remaining dataframe

In [ ]:
# Select only non-numeric columns
non_numeric_columns = df.select_dtypes(exclude=['number'])

# Select only numeric columns
numeric_columns = df.select_dtypes(include=['number'])

# get statistics for numeric columns
nan_counts_numeric = numeric_columns.isnull().sum()
unique_counts_numeric = numeric_columns.nunique()
percentage_nan_numeric = numeric_columns.isnull().mean() * 100


# create a new dataframe with the statistics for numeric columns
numeric_statistics = pd.DataFrame({'Nan Count': nan_counts_numeric,
                                    'Unique Count': unique_counts_numeric,
                                    'Percentage Nan': percentage_nan_numeric})


# print the numeric statistics dataframe
print(tabulate(numeric_statistics, headers='keys', tablefmt='fancy_grid'))

# get statistics for non-numeric columns
unique_counts_non_numeric = non_numeric_columns.nunique()
nan_counts_non_numeric = non_numeric_columns.isnull().sum()
percentage_nan_non_numeric = non_numeric_columns.isnull().mean() * 100

# create a new dataframe with the statistics for non-numeric columns
non_numeric_statistics = pd.DataFrame({'Nan Count': nan_counts_non_numeric,
                                       'Unique Count': unique_counts_non_numeric,
                                       'Percentage Nan': percentage_nan_non_numeric})

# print the non-numeric statistics dataframe
print(tabulate(non_numeric_statistics, headers='keys', tablefmt='fancy_grid'))

# Calculate shape
print("There are {} rows and {} columns in the dataset".format(df.shape[0], df.shape[1]))

Looking into the predicted variable

In [ ]:
# Get summary statistics for the 'sellingprice' column
sellingprice_stats = df['sellingprice'].describe()

# Convert the series to a DataFrame for tabulation
sellingprice_stats_df = pd.DataFrame({'Selling Price Stats': sellingprice_stats})

# Print the summary statistics using tabulate
print(tabulate(sellingprice_stats_df, headers='keys', tablefmt='fancy_grid'))

### 1.4 Correlation and descriptive analysis

Create a correlation heatmap

In [ ]:
# Compute the correlation matrix
corr_matrix = df.corr()

# Create a heatmap of the correlation matrix
sns.heatmap(corr_matrix, annot=True, cmap='Spectral')

# Show the plot
plt.show()

Inncluding the categorical features to check the correlation between the features

In [ ]:
# Create a copy of the DataFrame
df2 = df.copy()

# Factorize categorical columns to convert categorical data into numerical format
for column in df2.select_dtypes(exclude=[np.number]).columns:
     df2[column], labels = pd.factorize(df[column])
   
# Compute the correlation matrix
corr_matrix = df2.corr()

# Set the figure size
plt.figure(figsize=(12, 10))

# Create a heatmap of the correlation matrix
sns.clustermap(corr_matrix, annot=True, cmap='Spectral', fmt=".2f", vmin=-0.5, vmax=0.5)

# Show the plot
plt.show()

Based on the correaltion matrix above the following explanatory features are selected: 
year, condition, odometer, color and interior

Checking if there are outliers in the predicted column: selling_price using IQR

In [ ]:

# Calculate the first quartile (Q1) and third quartile (Q3)
Q1 = df['sellingprice'].quantile(0.25)
Q3 = df['sellingprice'].quantile(0.75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Print relevant values for debugging
print(f'- Q1: {Q1} | Q3: {Q3} | IQR: {IQR}')
print(f'- Lower Bound: {lower_bound} | Upper Bound: {upper_bound}')

# Identify outliers
outliers = df[(df['sellingprice'] < lower_bound) | (df['sellingprice'] > upper_bound)]

# Print the number of outliers
num_outliers = len(outliers)
print(f'- Number of outliers: {num_outliers}')

In [ ]:
# Get the number of rows before removing outliers
num_rows_before = len(df)

# Remove outliers from the 'sellingprice' column including the sellingprice = 1 
df = df[(df['sellingprice'] >= lower_bound) & (df['sellingprice'] <= upper_bound) & (df['sellingprice'] != 1)]

# Get the number of outliers removed
num_outliers_removed = num_rows_before - len(df)
print(f'Number of outliers removed: {num_outliers_removed}')

Checking if there are outliers in the odometer column using IQR

In [ ]:
# Calculate the first quartile (Q1) and third quartile (Q3)
Q1 = df['odometer'].quantile(0.25)
Q3 = df['odometer'].quantile(0.75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Print relevant values for debugging
print(f'- Q1: {Q1} | Q3: {Q3} | IQR: {IQR}')
print(f'- Lower Bound: {lower_bound} | Upper Bound: {upper_bound}')

# Identify outliers
outliers = df[(df['odometer'] < lower_bound) | (df['odometer'] > upper_bound)]

# Print the number of outliers
num_outliers = len(outliers)
print(f'- Number of outliers: {num_outliers}')

In [ ]:
# Get the number of rows before removing outliers
num_rows_before = len(df)

# Remove outliers from the 'sellingprice' column including the sellingprice = 1 
df = df[(df['odometer'] >= lower_bound) & (df['odometer'] <= upper_bound)]

# Get the number of outliers removed
num_outliers_removed = num_rows_before - len(df)
print(f'Number of outliers removed: {num_outliers_removed}')

Checking if there are outliers in the year column using IQR

In [ ]:
# Calculate the first quartile (Q1) and third quartile (Q3)
Q1 = df['year'].quantile(0.25)
Q3 = df['year'].quantile(0.75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Print relevant values for debugging
print(f'- Q1: {Q1} | Q3: {Q3} | IQR: {IQR}')
print(f'- Lower Bound: {lower_bound} | Upper Bound: {upper_bound}')

# Identify outliers
outliers = df[(df['year'] < lower_bound) | (df['year'] > upper_bound)]

# Print the number of outliers
num_outliers = len(outliers)
print(f'- Number of outliers: {num_outliers}')

In [ ]:
# Get the number of rows before removing outliers
num_rows_before = len(df)

# Remove outliers from the 'sellingprice' column including the sellingprice = 1 
df = df[(df['year'] >= lower_bound) & (df['year'] <= upper_bound)]

# Get the number of outliers removed
num_outliers_removed = num_rows_before - len(df)
print(f'Number of outliers removed: {num_outliers_removed}')

Adding a 'ID' column

In [ ]:
#Only keeping the columns which are needed 
df = df[['year', 'condition', 'odometer', 'color', 'interior', 'sellingprice']]
#Add an 'id' column with values starting from 1
df['ID']=range(1, len(df) +1) 

#Moving the 'id' column to the front
df.insert(0, 'ID', df.pop('ID'))

#Display the updated datframe 
print(tabulate(df.head(), headers='keys', tablefmt='fancy_grid'))
print(tabulate(df.tail(), headers='keys', tablefmt='fancy_grid'))

Analyzing the unique values of the explanatory features

In [ ]:
print(tabulate({
    'Year': df['year'].unique(),
    'Condition': df['condition'].unique(),
    'Odometer': df['odometer'].unique(),
    'Color': df['color'].unique(),
    'Interior': df['interior'].unique(),
 }, headers='keys', tablefmt='fancy_grid'))

Checking the value counts to identify the categories for the column "colors"

In [ ]:
print(tabulate(df['color'].value_counts().reset_index(), headers='keys', tablefmt='fancy_grid'))

Creating the categories for the column "color"

In [ ]:
# Define a function to categorize colors
def categorize_color(color):
     
    if 'black' in color:
        return 'black'
    elif 'white' in color:
        return 'white'
    elif 'silver' in color:
        return 'silver'
    elif 'gray' in color:
        return 'gray'
    else:
        return 'other'

# Apply the function to create a new 'color_category' column
df['color_cat'] = df['color'].apply(categorize_color)

# Display the count of unique values in 'color_category'
print(tabulate(df['color_cat'].value_counts().reset_index(), headers='keys', tablefmt='fancy_grid'))

Checking the value counts to identify the categories for the column "colors"


In [ ]:
print(tabulate(df['interior'].value_counts().reset_index(), headers='keys', tablefmt='fancy_grid'))

Creating categories for column 'interior'

In [ ]:
# Define a function to categorize interior
def categorize_interior(interior):
     
    if 'black' in interior:
        return 'black'
    # elif 'beige' in interior:
    #     return 'beige'
    elif 'gray' in interior:
        return 'gray'
    else:
        return 'other'

# Apply the function to create a new 'color_category' column
df['interior_cat'] = df['interior'].apply(categorize_interior)

# Display the count of unique values in 'color_category'
print(tabulate(df['interior_cat'].value_counts().reset_index(), headers='keys', tablefmt='fancy_grid'))

Dropping the color and interior column

In [ ]:
df.drop(['interior', 'color'], axis=1, inplace=True)

### 1.5 Distribution visualization

- Looking at the current distribution of sellingprice after cleaning the data
- And normalizing the data in this column

In [ ]:
# Apply square root transformation to 'sellingprice'
df['sqrt_sellingprice'] = np.sqrt(df['sellingprice'])

# Set a smaller figure size
plt.figure(figsize=(20, 10))

# Set font size for better readability
plt.rcParams.update({'font.size': 7})
# Create a histogram of the 'sellingprice' column
plt.subplot(1, 2, 1)
histplot = sns.histplot(df['sellingprice'], bins=20, kde=True, color='slateblue')

# Add frequency counts to each bin
for rect in histplot.patches:
    height = rect.get_height()
    width = rect.get_width()
    x = rect.get_x()
    y = rect.get_y()

    # Add the text annotation
    histplot.text(x + width / 2, y + height / 1.5 , f'{int(height)}', ha='center', va='center_baseline')

# Set plot titles and labels
plt.title('Distribution of Selling Prices')
plt.xlabel('Selling Price')
plt.ylabel('Frequency')

# Create a histogram of the 'sellingprice' column
plt.subplot(1, 2, 2)
histplot = sns.histplot(df['sqrt_sellingprice'], bins=20, kde=True, color='slateblue')

# Add frequency counts to each bin
for rect in histplot.patches:
    height = rect.get_height()
    width = rect.get_width()
    x = rect.get_x()
    y = rect.get_y()

    # Add the text annotation
    histplot.text(x + width / 2, y + height / 1.9 , f'{int(height)}', ha='center', va='center_baseline')

# Set plot titles and labels
plt.title('Distribution of Normalized Selling Prices')
plt.xlabel('Selling Price')
plt.ylabel('Frequency')

# Show the plot
plt.show()

Definine the function generate_freq_table to visualize the categorical plots

In [ ]:
# Group the DataFrame by the target variable and calculate the size of each group
def generate_freq_table(df, variable = ['sellingprice']):
    dfs = []
    for i in variable:
        df_count = (
            df.groupby(i, observed=False)
            .size()
            .reset_index(name='N')
            .assign(var = i)
            .rename(columns={i: 'category'})
        )
        # Append the result to the list of DataFrames
        dfs.append(df_count)
        # Concatenate all DataFrames in the list
        res = pd.concat(dfs)
        # Convert the 'category' column to string type
        res['category'] = res['category'].astype(str)
    # Return the final concatenated DataFrame
    return res

generate_freq_table(df, ['interior_cat', 'color_cat'])

Defining a function to create the plots

In [ ]:
def generate_freq_plot(freq_table):
    return (
        ggplot(freq_table, aes(x='var', y='N', fill='category')) +
        geom_col(stat='identity', position='dodge')
        )

Generating the interior categories plot

In [ ]:
generate_freq_plot(generate_freq_table(df, ['interior_cat']))

Generating the color categories plot

In [ ]:
generate_freq_plot(generate_freq_table(df, ['color_cat']))

Creating the plot for year

In [ ]:
# Generate the frequency plot of year with different colors
plt.figure(figsize=(10, 6))
histplot = sns.histplot(data=df, x='year', hue='year', bins=15, palette='viridis', kde=False)
plt.title('Frequency Plot of Years')
plt.xlabel('Year')
plt.ylabel('Frequency')
# Display plot
plt.show()

Creating the plot for condition

In [ ]:
# Generate the frequency plot of condition with different colors
plt.figure(figsize=(10, 6))
histplot = sns.histplot(data=df, x='condition', hue='condition', bins=41, palette='magma')
plt.title('Frequency Plot of Condition')
plt.xlabel('Year')
plt.ylabel('Frequency')
#Modify legend 
plt.legend(title='Condition', loc='upper left', ncol=2, labels= (df['condition'].unique()))

# Display plot
plt.show()

- Creating the plot for odometer
- And normalizing the data

In [ ]:
# Apply square root transformation to 'age'
df['sqrt_odometer'] = np.sqrt(df['odometer'])

# Set the figure size
plt.figure(figsize=(16, 6))  # Increased the figure width to accommodate both plots

# Create the first histogram of the 'odometer' column
plt.subplot(1, 2, 1)
sns.histplot(df['odometer'], bins=20, color='lightseagreen')
plt.title('Histogram of Odometer')
plt.xlabel('Odometer')
plt.ylabel('Frequency')

# Create the second histogram of a different column (replace 'another_column' with the actual column name)
plt.subplot(1, 2, 2)
sns.histplot(df['sqrt_odometer'], bins=20, color='lightseagreen')  # Replace 'another_column' with the actual column name
plt.title('Histogram of Normalized Odometer')
plt.xlabel('Odometer logged')
plt.ylabel('Frequency')

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()

## 2. Feature Engineering

Create a Trainclasssplitter class with 3 def fucntions:
- Innitialize instances
- Statistics calculator
- Train, test and validation splitter
It is important to note that the splitter splits based on an even distribution of the sellingprice target variable for each of the sets. This is done by transforming it to a categorical variable for the time being by creating sellingprice bins.

In [ ]:
class TrainTestSplitter(object):
    '''Class to perform the split of the data into train, test, and validation.'''
    def __init__(self, train_frac=0.8, validation_frac=0.2, seed=1234, num_bins=5):
        # Initialize the splitter with specified fractions, seed, and number of bins
        self.train_frac = train_frac
        self.validation_frac = validation_frac
        self.seed = seed
        self.num_bins = num_bins
        self.total_n_sellingprice_bin = {}  # Dictionary to store total counts for each sellingprice bin
    
    def calculate_statistics(self):
        # Calculate statistics for each split (train, test, validation)
        statistics = {}
        for i in ['train_set', 'test_set', 'validation_set']:
            split_stats = {}
            
            # Create bins for sellingprice in the current split
            bins = np.linspace(getattr(self, i)['sellingprice'].min(), getattr(self, i)['sellingprice'].max(), self.num_bins + 1)
            getattr(self, i)['sellingprice_bin'] = pd.cut(getattr(self, i)['sellingprice'], bins=bins)
            
            # Count the occurrences of each sellingprice bin in the current split
            target_count = getattr(self, i).groupby('sellingprice_bin').size().reset_index()
            
            for _, row in target_count.iterrows():
                bin_label = str(row['sellingprice_bin'])
                bin_count = row[0]
                percentage_key = f'percentage_total_sellingprice_bin_{bin_label}'
                
                # Store counts and percentages in the split_stats dictionary
                split_stats[f'N_sellingprice_bin_{bin_label}'] = bin_count
                split_stats[percentage_key] = bin_count / self.total_n_sellingprice_bin.get(bin_label, 1) * 100

            statistics[i] = split_stats

        self.split_statistics = statistics

    def split_train_test(self, df):
        print("Generating the train/validation/test splits...")
        
        # Create bins for sellingprice in the entire dataset
        bins = np.linspace(df['sellingprice'].min(), df['sellingprice'].max(), self.num_bins + 1)
        df['sellingprice_bin'] = pd.cut(df['sellingprice'], bins=bins)
        
        # Count the total occurrences of each sellingprice bin in the entire dataset
        for bin_label in df['sellingprice_bin'].unique():
            bin_count = df.loc[lambda x: x.sellingprice_bin == bin_label].shape[0]
            self.total_n_sellingprice_bin[str(bin_label)] = bin_count

        # Sample the training set using the specified fraction and seed
        self.train_set = df.sample(frac=self.train_frac, random_state=self.seed)
        # Create the test set by excluding IDs present in the training set
        self.test_set = df.loc[lambda x: ~x.ID.isin(self.train_set.ID)].reset_index(drop=True)
        # Sample the validation set from the training set
        self.validation_set = self.train_set.sample(frac=self.validation_frac).reset_index(drop=True)
        # Exclude validation set IDs from the training set
        self.train_set = self.train_set.loc[lambda x: ~x.ID.isin(self.validation_set.ID)].reset_index(drop=True)
        
        print("Calculating the statistics...")
        # Calculate and store statistics for the splits
        self.calculate_statistics()
        print("Split completed")


Now, its time to perform the splits

In [ ]:
# Create a fitting_splits object that will hold the train, validation, and test data
fitting_splits = TrainTestSplitter()
fitting_splits.split_train_test(df)

Now some statistics of the split. The goal is to have an even percentage of the data per bin. These percentages should be similar per bin.

In [ ]:
fitting_splits.test_set.shape
print(tabulate(fitting_splits.split_statistics, headers="keys", tablefmt= "fancy_grid"))

Next, create 3 def functions:
- Dummificator
- Scaler
- A function that uses the other 2 def functions

The code takes onehot encoded features into consideration and makes sure that these columns are not used for the scaler

In [ ]:
# Def fucntion for dummification
def dummify(df, one_hot_encoder):
    # Specify the variables to encode
    vars_to_encode = ['interior_cat', 'color_cat']
    
    # Extract the columns to be encoded
    df_to_encode = df[vars_to_encode]
    
    # Initialize or use the provided OneHotEncoder
    if not one_hot_encoder:
        one_hot_encoder = OneHotEncoder()
        df_encoded = one_hot_encoder.fit_transform(df_to_encode).toarray()
    else:
        df_encoded = one_hot_encoder.transform(df_to_encode).toarray()
    
    # Create a DataFrame from the encoded columns
    df_encoded = pd.DataFrame(df_encoded, columns=one_hot_encoder.get_feature_names_out())
    
    # Add the encoded columns and drop the original columns
    df = pd.concat([df, df_encoded], axis=1)
    df = df.drop(vars_to_encode, axis=1)
    
    return df, one_hot_encoder

# Def fucntion for standard scaling
def scale(df, standard_scaler, cols_to_scale):
    # Initialize or use the provided StandardScaler
    if not standard_scaler:
        standard_scaler = StandardScaler()
        df[cols_to_scale] = standard_scaler.fit_transform(df[cols_to_scale])
    else:
        df[cols_to_scale] = standard_scaler.transform(df[cols_to_scale])
    
    return df, standard_scaler

# Def function for data preperation
def prepare_data(df, one_hot_encoder=None, standard_scaler=None, cols_to_scale=None):
    # Reset the index of the DataFrame
    df = df.reset_index(drop=True)
    
    # Perform one-hot encoding
    df, one_hot_encoder = dummify(df, one_hot_encoder)
    
    # Identify columns to scale (numerical features)
    if cols_to_scale is None:
        cols_to_scale = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Identify columns to exclude from scaling (one-hot encoded columns)
    one_hot_columns = one_hot_encoder.get_feature_names_out() if one_hot_encoder else []
    cols_to_exclude = df.columns[df.columns.isin(one_hot_columns)]
    
    # Remove one-hot encoded columns from the list of columns to scale
    cols_to_scale = list(set(cols_to_scale) - set(cols_to_exclude))
    
    # Perform feature scaling
    df, standard_scaler = scale(df, standard_scaler, cols_to_scale)
    
    return df, one_hot_encoder, standard_scaler


## 3. Datasets generation

Inhere, 8 datasets are made:
- X_train 
- y_train
- x_validation
- y_validation
- x_test
- y_test
- x_train_validation
- y_train_validation

The dataset 'x_train_validation' and 'y_train_validation' are concatenations of the train and validation sets.

For every dataset, the onehotencoder, scaler and required features are applied.

In [ ]:
# Prepare all the data for subsequent use
# Training Set
X_train, one_hot_encoder, standard_scaler = prepare_data(fitting_splits.train_set)

# Drop unnecessary columns from the training set
X_train = X_train.drop(["ID", 'odometer', 'sellingprice', 'sqrt_sellingprice', 'sellingprice_bin'], axis=1)

# Extract the target variable for the training set (square root of selling price)
y_train = fitting_splits.train_set["sqrt_sellingprice"]

# Validation Set
# Use the same one-hot encoder and standard scaler as the training set
X_validation = prepare_data(fitting_splits.validation_set, one_hot_encoder, standard_scaler)[0]

# Drop unnecessary columns from the validation set
X_validation = X_validation.drop(["ID", 'odometer', 'sellingprice', 'sqrt_sellingprice', 'sellingprice_bin'], axis=1)

# Extract the target variable for the validation set (square root of selling price)
y_validation = fitting_splits.validation_set["sqrt_sellingprice"]

# Test Set
# Use the same one-hot encoder and standard scaler as the training set
X_test = prepare_data(fitting_splits.test_set, one_hot_encoder, standard_scaler)[0]

# Drop unnecessary columns from the test set
X_test = X_test.drop(['sellingprice_bin', "ID", 'odometer', 'sellingprice', 'sqrt_sellingprice'], axis=1)

# Extract the target variable for the test set (square root of selling price)
y_test = fitting_splits.test_set["sqrt_sellingprice"]

# Combine Training and Validation Sets for Cross-Validation
X_train_validation = pd.concat([X_train, X_validation])
y_train_validation = pd.concat([y_train, y_validation])


## 4. Model fitting: Random Forest

The model is fitted trying to find the best model in a range of hyperparameters. The best model is based on the lowest MSE score, instead of the f1 score for classification problems

It uses 5-fold cross-validation to find the best hyperparameters by doing the k-fold cross-validation calculations without using sklearn liberaries. 

A pickle is saved to avoid resoing the grid search process.

A pickle has been made of the outcomes of the fit

In [ ]:
# Check if the pickle file already exists
pickle_filename = 'grid_search_rf_model.pkl'
if os.path.exists(pickle_filename):
    # Load the existing GridSearchCV object
    with open(pickle_filename, 'rb') as file:
        sklearn_grid_search_rf = pickle.load(file)
else:
    # Define the parameter grid for GridSearchCV
    param_grid = {
        "n_estimators": [x for x in range(10, 200, 10)],
        "max_depth": [x for x in range(5, 21, 5)]
    }

    # Create a RandomForestRegressor
    rf_model = RandomForestRegressor()

    # Create a GridSearchCV object
    sklearn_grid_search_rf = GridSearchCV(rf_model, param_grid, cv=5)

    # Fit the GridSearchCV object to your data
    sklearn_grid_search_rf.fit(X_train_validation, y_train_validation)

    # Save the GridSearchCV object
    with open(pickle_filename, 'wb') as file:
        pickle.dump(sklearn_grid_search_rf, file)

    # Initialize lists to store results
    mse_scores = []

    # Define the number of folds for cross-validation
    num_folds = 5

    # Get the number of samples in your dataset
    num_samples = X_train_validation.shape[0]

    # Calculate the size of each fold
    fold_size = num_samples // num_folds

    # Iterate over the folds
    for fold in range(num_folds):
        # Define the indices for the current fold
        start_idx = fold * fold_size
        end_idx = (fold + 1) * fold_size if fold < num_folds - 1 else num_samples
        
        # Create training and validation sets for the current fold
        X_train_fold = np.concatenate([X_train_validation[:start_idx], X_train_validation[end_idx:]])
        y_train_fold = np.concatenate([y_train_validation[:start_idx], y_train_validation[end_idx:]])
        
        X_val_fold = X_train_validation[start_idx:end_idx]
        y_val_fold = y_train_validation[start_idx:end_idx]

        # Create a RandomForestRegressor
        rf_model = RandomForestRegressor()

        # Set up the model with specified parameters
        rf_model.set_params(**sklearn_grid_search_rf.best_params_)

        # Fit the model on the training fold
        rf_model.fit(X_train_fold, y_train_fold)

        # Make predictions on the validation fold
        y_pred = rf_model.predict(X_val_fold)

        # Calculate mean squared error for the fold
        mse_fold = np.mean((y_val_fold - y_pred) ** 2)

        # Append the MSE for the fold to the list
        mse_scores.append(mse_fold)

    # Calculate the average MSE across all folds
    average_mse = np.mean(mse_scores)

    print("Average Mean Squared Error (MSE) across 5 folds:", average_mse)

The average MSE across each of the folds is 459

## 5. Model evaluation: Random Forest

Evaluation of the model can not be done based of the f1 score since it is a regression problem and not a classification problem. Therefore, the MSE is used as metric of the model.

In [ ]:
# Print the best parameters found by the grid search
print("Best Parameters: ", sklearn_grid_search_rf.best_params_)

# Retrieve the best model from the grid search
best_model = sklearn_grid_search_rf.best_estimator_

# Print the best cross-validated score (negative mean squared error) obtained during the grid search
print("Best Cross-Validated Score (MSE): ", -sklearn_grid_search_rf.best_score_)

# Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

# Calculate the mean squared error on the test set
mse = mean_squared_error(y_test, y_pred)

# Print the mean squared error on the test set
print("Test Set MSE: ", mse)

The test set MSE is as good as equal to the cross validation MSE.

We can conclude that the model shows:
- Consistent Performance:

If the test set MSE is close to the cross-validated MSE, it could indicate that your model is generalizing well to unseen data. This consistency suggests that the performance observed during cross-validation is a good reflection of how the model will perform on new, unseen samples.

- Appropriate Model Complexity:

The fact that the best hyperparameters from the cross-validation also perform well on the test set suggests that the chosen model complexity (determined by hyperparameters like max_depth and n_estimators in the case of a RandomForestRegressor) is appropriate for the problem at hand.

- Reliable Evaluation:

The cross-validated MSE serves as an estimate of the model's expected performance on new data. When the test set performance aligns with the cross-validated performance, it adds confidence to the reliability of the cross-validated estimate.

- No Overfitting:

If the test set MSE was significantly higher than the cross-validated MSE, it might indicate overfitting, where the model performs well on the training data but poorly on new data. The fact that they are comparable suggests that overfitting is less of a concern.

Let's create two plots to visualize our model's performance. The first plot compares the actual values with the predicted values. The second plot compares the residuals.

In [ ]:
# Calculate residuals by subtracting predicted values from actual values
residuals = y_test - y_pred

# Scatter Plot: Visualizing predicted vs. actual selling prices with residuals as color
plt.scatter(y_test, y_pred, alpha=0.5, s=10, c=residuals, cmap='coolwarm')
plt.xlabel("Actual Selling Prices")
plt.ylabel("Predicted Selling Prices")
plt.title("Actual vs. Predicted Selling Prices")

# Add a colorbar to indicate the magnitude of residuals
cbar = plt.colorbar()
cbar.set_label('Residuals')

# Residual Plot: Visualizing residuals against actual selling prices
plt.figure()
plt.scatter(y_test, residuals, alpha=0.5, s=10, c=residuals, cmap='coolwarm')
plt.axhline(y=0, color='r', linestyle='--')  # Add a horizontal line at y=0 for reference
plt.xlabel("Actual Selling Prices")
plt.ylabel("Residuals")
plt.title("Residual Plot")

# Add a colorbar to indicate the magnitude of residuals
cbar = plt.colorbar()
cbar.set_label('Residuals')

# Display the plots
plt.show()

So our Test Set MSE is 459, but what does this mean? A good way to evaluate this is by comparing it to the variance of our target variable sellingprice. Please keep in mind that this is the squareroot version of sellingprice.

In [ ]:
# Calculate Mean Squared Error (MSE)
mse_rf = mean_squared_error(y_test, y_pred)

# Calculate the Variance of the Target Variable
variance = np.var(y_test)

# Calculate R-squared, a measure of how well the model explains the variance in the target variable
r_squared_rf = r2_score(y_test, y_pred)

# Calculate the Ratio of MSE to Variance, a metric to assess model performance
mse_to_variance_ratio_rf = mse_rf / variance

# Print the calculated metrics
print(f'Mean Squared Error (MSE): {mse_rf}')
print(f'Variance of the Target Variable: {variance}')
print(f'R-squared: {r_squared_rf}')
print(f'MSE to Variance Ratio: {mse_to_variance_ratio_rf}')

Let's look at the outcomes

- Mean Squared Error (MSE): 459

An MSE of 459.10 indicates that, on average, the squared difference between your predicted and actual values is relatively moderate. The mean error is around 21 dollar, but since this is based on squareroot sellingprice, we have to square it again. So the average discrepancy of a prediction is 459 dollars, which is a reasonable outcome

- Variance of the Target Variable: 1213.13

In comparison to the variance, the MSE is smaller, suggesting that the model is capturing a significant portion of the variability.

- R-squared: 0.62

The R-squared value of 0.62 indicates that the model explains approximately 62% of the variability in the target variable. This is a moderate level of explanatory power.

- MSE to Variance Ratio: 0.38

The MSE to Variance Ratio is less than 1, which suggests that the model is performing reasonably well. A ratio closer to 0 indicates that the model is capturing a substantial portion of the variance relative to the inherent variability in the target variable.

## 6. Model Fitting: ANN

First we want to know the amount of neurons in the input layer

In [ ]:
print(f"The number of neurons for the input layer must be: {X_train.shape[1]}")

Now we create a model

In [ ]:
# function that creates the model
# note that we pass the number of neurons as a parameter to the network
def create_model(neurons=10):
    nn_model = Sequential()
    nn_model.add(Dense(neurons, input_dim=X_train.shape[1], activation="relu"))
    nn_model.add(Dense(1))
    nn_model.compile(loss='mean_squared_error', optimizer='adam')
    return nn_model

seed = 1234
np.random.seed(seed)

# turn the keras model into a sklearn compatible model
# note that the neurons parameter needs to be specified in the interface of KerasClassifier
model = KerasRegressor(build_fn=create_model, verbose=0)

We set the hyperparameters that can be used for grid search

In [ ]:
# define the grid search parameters
batch_size = [10, 20]
epochs = [100, 150]
neurons = [10, 20, 30]
params_grid = dict(batch_size=batch_size, epochs=epochs, neurons=neurons)

We perform grid search for hyperparameter tuning and save it in a pickle, if the pickle already exists this step is not repeated

In [ ]:
# Define the pickle file path
pickle_file_path = 'nn_grid_search.pkl'

# perform grid search with sklearn if needed, otherwise load the grid search already performed
if os.path.exists(pickle_file_path):
    with open(pickle_file_path, 'rb') as handle:
        grid_search_nn = pickle.load(handle)
else:
    start_time = datetime.datetime.now()

    def create_model(neurons=10):
        nn_model = Sequential()
        nn_model.add(Dense(neurons, input_dim=X_train_validation.shape[1], activation="relu"))
        nn_model.add(Dense(1))
        nn_model.compile(loss='mean_squared_error', optimizer='adam')
        return nn_model

    model = KerasRegressor(neurons=10, build_fn=create_model, verbose=0)
    
    # Create GridSearchCV with the modified model and parameter grid
    grid_search_nn = GridSearchCV(estimator=model, param_grid=params_grid, n_jobs=-1, cv=5)
    grid_search_nn = grid_search_nn.fit(X_train_validation, y_train_validation)
    
    end_time = datetime.datetime.now()
    print(f'hypertuning with sklearn grid search for neural networks complete in {round((end_time - start_time).seconds/60, 2)} minutes')
    
    # store the results of the grid search to disk
    with open(pickle_file_path, 'wb') as handle:
        pickle.dump(grid_search_nn, handle)

## 7. Model evaluation: ANN

Finding best parameters

In [ ]:
# Print or return the results
print("Best Parameters:", grid_search_nn.best_params_)
print("Best Cross-Validated Score (MSE):", -grid_search_nn.best_score_)

Calculating test set MSE

In [ ]:
# Define the best model
best_model = grid_search_nn.best_estimator_

# Predict y based on test set
y_pred = best_model.predict(X_test)

# Calculate the MSE
mse_test = mean_squared_error(y_test, y_pred)
print("Test Set MSE:", mse_test)


All results for each parameter combination

In [ ]:
all_results = grid_search_nn.cv_results_
print("Grid Search Results:")
for params, mean_score, std_score in zip(all_results['params'], all_results['mean_test_score'], all_results['std_test_score']):
    print(f"Parameters: {params}, Mean Score (MSE): {-mean_score}, Standard Deviation: {std_score}")

Let's compare the MSE to the variance again

In [ ]:
# Calculate Mean Squared Error (MSE)
mse_ann = mean_squared_error(y_test, y_pred)

# Calculate the Variance of the Target Variable
variance = np.var(y_test)

# Calculate R-squared, a measure of how well the model explains the variance in the target variable
r_squared_ann = r2_score(y_test, y_pred)

# Calculate the Ratio of MSE to Variance, a metric to assess model performance
mse_to_variance_ratio_ann = mse_ann / variance

# Print the calculated metrics
print(f'Mean Squared Error (MSE): {mse_ann}')
print(f'Variance of the Target Variable: {variance}')
print(f'R-squared: {r_squared_ann}')
print(f'MSE to Variance Ratio: {mse_to_variance_ratio_ann}')

- Mean Squared Error (MSE): 460.49

This 460.49 suggests a moderate level of prediction error. The interpretation of MSE depends on the scale of your target variable.

- Variance of the Target Variable: 1213.13

- R-squared: 0.6204

R-squared of 0.62 indicates that your model explains approximately 62% of the variability in the target variable.

- MSE to Variance Ratio: 0.3796

The ratio is 0.38, suggesting that the prediction error is relatively lower compared to the variance of the target variable.

Let's visualize the results

In [ ]:
# Calculate residuals by subtracting predicted values from actual values
residuals = y_test - y_pred

# Scatter Plot: Visualizing predicted vs. actual selling prices with residuals as color
plt.scatter(y_test, y_pred, alpha=0.5, s=10, c=residuals, cmap='coolwarm')
plt.xlabel("Actual Selling Prices")
plt.ylabel("Predicted Selling Prices")
plt.title("Actual vs. Predicted Selling Prices")

# Add a colorbar to indicate the magnitude of residuals
cbar = plt.colorbar()
cbar.set_label('Residuals')

# Residual Plot: Visualizing residuals against actual selling prices
plt.figure()
plt.scatter(y_test, residuals, alpha=0.5, s=10, c=residuals, cmap='coolwarm')
plt.axhline(y=0, color='r', linestyle='--')  # Add a horizontal line at y=0 for reference
plt.xlabel("Actual Selling Prices")
plt.ylabel("Residuals")
plt.title("Residual Plot")

# Add a colorbar to indicate the magnitude of residuals
cbar = plt.colorbar()
cbar.set_label('Residuals')

# Display the plots
plt.show()

## 8. Model fitting: XGBoost

The model is fit and saved in a pickle, unless the pickle already exists

In [ ]:
# Define the pickle file path
pickle_file_path = 'grid_search_xgb_model.pkl'

# Check if the pickle file already exists
if os.path.exists(pickle_file_path):
    # Load the existing GridSearchCV object
    with open(pickle_file_path, 'rb') as file:
        grid_search = pickle.load(file)
else:
    # Create an XGBoost regressor
    xg_reg = xgb.XGBRegressor(objective='reg:squarederror')

    # Define a parameter grid to search
    param_grid = {
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'n_estimators': [50, 100, 200],
        'colsample_bytree': [0.5, 0.7, 0.9],
    }

    # Create the GridSearchCV object
    grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)

    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)

    # Save the GridSearchCV object (including best model) to a pickle file
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(grid_search, file)

## 9. Model evaluation: XGBoost

The best parameters are evaluated

In [ ]:
# Print or return the results
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validated Score (MSE):", -grid_search.best_score_)

The testset MSE is calculated

In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print("Test Set MSE:", mse_test)


The resuts are again compared with the variance

In [ ]:
# Calculate Mean Squared Error (MSE)
mse_xgb = mean_squared_error(y_test, y_pred)

# Calculate the Variance of the Target Variable
variance = np.var(y_test)

# Calculate R-squared, a measure of how well the model explains the variance in the target variable
r_squared_xgb = r2_score(y_test, y_pred)

# Calculate the Ratio of MSE to Variance, a metric to assess model performance
mse_to_variance_ratio_xgb = mse_xgb / variance

# Print the calculated metrics
print(f'Mean Squared Error (MSE): {mse_xgb}')
print(f'Variance of the Target Variable: {variance}')
print(f'R-squared: {r_squared_xgb}')
print(f'MSE to Variance Ratio: {mse_to_variance_ratio_xgb}')

Again two plots are made

In [ ]:
# Calculate residuals by subtracting predicted values from actual values
residuals = y_test - y_pred

# Scatter Plot: Visualizing predicted vs. actual selling prices with residuals as color
plt.scatter(y_test, y_pred, alpha=0.5, s=10, c=residuals, cmap='coolwarm')
plt.xlabel("Actual Selling Prices")
plt.ylabel("Predicted Selling Prices")
plt.title("Actual vs. Predicted Selling Prices")

# Add a colorbar to indicate the magnitude of residuals
cbar = plt.colorbar()
cbar.set_label('Residuals')

# Residual Plot: Visualizing residuals against actual selling prices
plt.figure()
plt.scatter(y_test, residuals, alpha=0.5, s=10, c=residuals, cmap='coolwarm')
plt.axhline(y=0, color='r', linestyle='--')  # Add a horizontal line at y=0 for reference
plt.xlabel("Actual Selling Prices")
plt.ylabel("Residuals")
plt.title("Residual Plot")

# Add a colorbar to indicate the magnitude of residuals
cbar = plt.colorbar()
cbar.set_label('Residuals')

# Display the plots
plt.show()

## 10. Model fitting: KNN

The KNN model is fitted and again saved in a pickle

In [ ]:
# File path for the pickle file
pickle_file_path = 'grid_search_knn_model.pkl'

# Check if the pickle file already exists
if os.path.exists(pickle_file_path):
    # Load the GridSearchCV object from the pickle file
    with open(pickle_file_path, 'rb') as file:
        grid_search = pickle.load(file)
else:
    # Create a KNeighborsRegressor
    knn_reg = KNeighborsRegressor()

    # Define a parameter grid to search
    param_grid = {
        'n_neighbors': [x for x in range(1, 40)],
        'weights': ['uniform', 'distance'],
        'p': [1, 2],  # 1 for Manhattan distance, 2 for Euclidean distance
        'metric': ['minkowski', 'cosine']
    }

    # Create the GridSearchCV object
    grid_search = GridSearchCV(estimator=knn_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)

    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)

    # Save the GridSearchCV object (including the best model) to a pickle file
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(grid_search, file)

## 11. Model evaluation: KNN

What are the best parameters and the MSE?

In [ ]:
# Print or return the results
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validated Score (MSE):", -grid_search.best_score_)

What is the test set MSE?

In [ ]:
# Define best model
best_model = grid_search.best_estimator_

# Predict y
y_pred = best_model.predict(X_test)

# Calculate MSE of test set
mse_test = mean_squared_error(y_test, y_pred)
print("Test Set MSE:", mse_test)

Evaluate results of model

In [ ]:
# Calculate Mean Squared Error (MSE)
mse_knn = mean_squared_error(y_test, y_pred)

# Calculate the Variance of the Target Variable
variance = np.var(y_test)

# Calculate R-squared, a measure of how well the model explains the variance in the target variable
r_squared_knn = r2_score(y_test, y_pred)

# Calculate the Ratio of MSE to Variance, a metric to assess model performance
mse_to_variance_ratio_knn = mse_knn / variance

# Print the calculated metrics
print(f'Mean Squared Error (MSE): {mse_knn}')
print(f'Variance of the Target Variable: {variance}')
print(f'R-squared: {r_squared_knn}')
print(f'MSE to Variance Ratio: {mse_to_variance_ratio_knn}')

Visualize results

In [ ]:
# Calculate residuals by subtracting predicted values from actual values
residuals = y_test - y_pred

# Scatter Plot: Visualizing predicted vs. actual selling prices with residuals as color
plt.scatter(y_test, y_pred, alpha=0.5, s=10, c=residuals, cmap='coolwarm')
plt.xlabel("Actual Selling Prices")
plt.ylabel("Predicted Selling Prices")
plt.title("Actual vs. Predicted Selling Prices")

# Add a colorbar to indicate the magnitude of residuals
cbar = plt.colorbar()
cbar.set_label('Residuals')

# Residual Plot: Visualizing residuals against actual selling prices
plt.figure()
plt.scatter(y_test, residuals, alpha=0.5, s=10, c=residuals, cmap='coolwarm')
plt.axhline(y=0, color='r', linestyle='--')  # Add a horizontal line at y=0 for reference
plt.xlabel("Actual Selling Prices")
plt.ylabel("Residuals")
plt.title("Residual Plot")

# Add a colorbar to indicate the magnitude of residuals
cbar = plt.colorbar()
cbar.set_label('Residuals')

# Display the plots
plt.show()

## 12. Model fitting: Linear Regression

Fit linear regression model

In [ ]:
# Create a linear regression model
linear_reg_model = LinearRegression()

# Fit the model to the training data
linear_reg_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = linear_reg_model.predict(X_test)

## 13. Model evaluation: Linear Regression

Calculate MSE and R2-score

In [ ]:
# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("Mean Squared Error (MSE):", mse)
print("R-squared:", r2)

Evaluate results

In [ ]:
# Calculate Mean Squared Error (MSE)
mse_lr = mean_squared_error(y_test, y_pred)

# Calculate the Variance of the Target Variable
variance = np.var(y_test)

# Calculate R-squared, a measure of how well the model explains the variance in the target variable
r_squared_lr = r2_score(y_test, y_pred)

# Calculate the Ratio of MSE to Variance, a metric to assess model performance
mse_to_variance_ratio_lr = mse_lr / variance

# Print the calculated metrics
print(f'Mean Squared Error (MSE): {mse_lr}')
print(f'Variance of the Target Variable: {variance}')
print(f'R-squared: {r_squared_lr}')
print(f'MSE to Variance Ratio: {mse_to_variance_ratio_lr}')

Visualize results

In [ ]:
# Calculate residuals by subtracting predicted values from actual values
residuals = y_test - y_pred

# Scatter Plot: Visualizing predicted vs. actual selling prices with residuals as color
plt.scatter(y_test, y_pred, alpha=0.5, s=10, c=residuals, cmap='coolwarm')
plt.xlabel("Actual Selling Prices")
plt.ylabel("Predicted Selling Prices")
plt.title("Actual vs. Predicted Selling Prices")

# Add a colorbar to indicate the magnitude of residuals
cbar = plt.colorbar()
cbar.set_label('Residuals')

# Residual Plot: Visualizing residuals against actual selling prices
plt.figure()
plt.scatter(y_test, residuals, alpha=0.5, s=10, c=residuals, cmap='coolwarm')
plt.axhline(y=0, color='r', linestyle='--')  # Add a horizontal line at y=0 for reference
plt.xlabel("Actual Selling Prices")
plt.ylabel("Residuals")
plt.title("Residual Plot")

# Add a colorbar to indicate the magnitude of residuals
cbar = plt.colorbar()
cbar.set_label('Residuals')

# Display the plots
plt.show()

## 14. Evaluation of models

Create a table of all models results ranked on their MSE to variance ratio

In [ ]:
# Results for Random Forest
results_rf = ["Random Forest", mse_rf, variance, r_squared_rf, mse_to_variance_ratio_rf]

# Results for ANN
results_ann = ["Artificial Neural Network", mse_ann, variance, r_squared_ann, mse_to_variance_ratio_ann]

# Results for XGBoost
results_xgb = ["XGBoost", mse_xgb, variance, r_squared_xgb, mse_to_variance_ratio_xgb]

# Results for Linear Regression
results_lr = ["Linear Regression", mse_lr, variance, r_squared_lr, mse_to_variance_ratio_lr]

# Results for KNN
results_knn = ["K-Nearest Neighbors", mse_knn, variance, r_squared_knn, mse_to_variance_ratio_knn]

# Create a list of results
all_results = [results_rf, results_ann, results_xgb, results_lr, results_knn]

# Define column headers
headers = ["Model", "Mean Squared Error (MSE)", "Variance", "R-squared", "MSE to Variance Ratio"]

# Sort the results based on MSE to Variance Ratio
all_results_sorted = sorted(all_results, key=lambda x: x[-1])  # Assuming the MSE to Variance Ratio is in the last column

# Create the table
table = tabulate(all_results_sorted, headers, tablefmt="fancy_grid")

# Print the table
print(table)

Visualize the results

In [ ]:
# Assuming all_results_sorted from the previous example
models = [result[0] for result in all_results_sorted]
mse_to_variance_ratio = [result[-1] for result in all_results_sorted]

# Bar Chart
plt.figure(figsize=(8, 6))
sns.barplot(x=models, y=mse_to_variance_ratio, palette='viridis', width=0.3)
plt.xlabel('Model')
plt.ylabel('MSE to Variance Ratio')
plt.title('Model Comparison based on MSE to Variance Ratio')
plt.xticks(rotation=45, ha='right')
plt.ylim(0.35, max(mse_to_variance_ratio) + 0.01)  # Set y-axis limit
plt.tight_layout()
plt.show()